In [34]:
import pandas as pd
import numpy as np
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import GPT2TokenizerFast, GPT2ForTokenClassification, AdamW
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
import warnings
warnings.filterwarnings('ignore')

# Stałe
BATCH_SIZE = 16
EPOCHS = 3
LEARNING_RATE = 2e-5
MAX_LEN = 128
TRAIN_SIZE = 0.8


In [35]:

# Wczytanie i przygotowanie danych
df = pd.read_csv('data/annotations_all_batches - WORD - SECOND BATCH.csv')
df = df.fillna(method='ffill')

# Grupowanie po sentence_id
sentences = df.groupby('sentence_id').agg({
    'word': lambda x: list(x),
    'final-annotation': lambda x: list(x)
}).reset_index()


In [36]:
# Klasa dataset
class TokenClassificationDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, max_len):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_len = max_len
        
    def __len__(self):
        return len(self.texts)
    
    def __getitem__(self, idx):
        words = self.texts[idx]
        labels = self.labels[idx]
        
        # Tokenizacja
        encoding = self.tokenizer(
            words,
            is_split_into_words=True,
            max_length=self.max_len,
            padding='max_length',
            truncation=True,
            return_tensors='pt'
        )
        
        # Dostosowanie etykiet do tokenów
        word_ids = encoding.word_ids()
        label_ids = []
        
        for word_id in word_ids:
            if word_id is None:
                label_ids.append(-100)
            else:
                label_ids.append(labels[word_id])
                
        return {
            'input_ids': encoding['input_ids'].flatten(),
            'attention_mask': encoding['attention_mask'].flatten(),
            'labels': torch.tensor(label_ids)
        }


In [37]:

# Przygotowanie danych
texts = sentences['word'].values
labels = sentences['final-annotation'].values

# Podział na zbiór treningowy i testowy
train_texts, test_texts, train_labels, test_labels = train_test_split(
    texts, labels, train_size=TRAIN_SIZE, random_state=42
)


In [ ]:

# Inicjalizacja tokenizera i modelu
tokenizer = GPT2TokenizerFast.from_pretrained('gpt2', add_prefix_space=True)
tokenizer.pad_token = tokenizer.eos_token  # Ustawienie tokenu pad jako eos dla GPT-2
# przypisać -100 tam gdzie jest eos
model = GPT2ForTokenClassification.from_pretrained(
    'gpt2',
    num_labels=4
)


Some weights of GPT2ForTokenClassification were not initialized from the model checkpoint at gpt2 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [42]:

# Przygotowanie datasetów
train_dataset = TokenClassificationDataset(train_texts, train_labels, tokenizer, MAX_LEN)
test_dataset = TokenClassificationDataset(test_texts, test_labels, tokenizer, MAX_LEN)

train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE)


In [50]:

# Trening modelu
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)
optimizer = AdamW(model.parameters(), lr=LEARNING_RATE)

for epoch in range(500):
    model.train()
    total_loss = 0
    
    for batch in train_loader:
        optimizer.zero_grad()
        
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)
        
        outputs = model(
            input_ids=input_ids,
            attention_mask=attention_mask,
            labels=labels
        )
        
        loss = outputs.loss
        total_loss += loss.item()
        
        loss.backward()
        optimizer.step()
    
    print(f'Epoch {epoch + 1}, Loss: {total_loss/len(train_loader)}')


Epoch 1, Loss: 0.030292129144072533
Epoch 2, Loss: 0.03399297967553139
Epoch 3, Loss: 0.039301272481679916
Epoch 4, Loss: 0.0375821627676487
Epoch 5, Loss: 0.03289886564016342
Epoch 6, Loss: 0.025759031996130943
Epoch 7, Loss: 0.04060889407992363
Epoch 8, Loss: 0.034197308123111725
Epoch 9, Loss: 0.03520038351416588
Epoch 10, Loss: 0.05086660757660866
Epoch 11, Loss: 0.04610062390565872
Epoch 12, Loss: 0.03443412110209465
Epoch 13, Loss: 0.03022284060716629
Epoch 14, Loss: 0.043960586190223694
Epoch 15, Loss: 0.041107144206762314
Epoch 16, Loss: 0.02598210796713829
Epoch 17, Loss: 0.031245170161128044
Epoch 18, Loss: 0.03085775300860405
Epoch 19, Loss: 0.04445182532072067
Epoch 20, Loss: 0.017824500799179077
Epoch 21, Loss: 0.025048833340406418
Epoch 22, Loss: 0.023432299494743347
Epoch 23, Loss: 0.03064490109682083
Epoch 24, Loss: 0.023311764001846313
Epoch 25, Loss: 0.016738664358854294
Epoch 26, Loss: 0.026064004749059677
Epoch 27, Loss: 0.01835688389837742
Epoch 28, Loss: 0.0163110

In [51]:

# Ewaluacja
model.eval()
predictions = []
true_labels = []

with torch.no_grad():
    for batch in test_loader:
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)
        
        outputs = model(
            input_ids=input_ids,
            attention_mask=attention_mask
        )
        
        preds = torch.argmax(outputs.logits, dim=2)
        
        for i in range(len(preds)):
            pred = preds[i][batch['attention_mask'][i] == 1]
            label = labels[i][batch['attention_mask'][i] == 1]
            
            pred = pred[label != -100]
            label = label[label != -100]
            
            predictions.extend(pred.cpu().numpy())
            true_labels.extend(label.cpu().numpy())

# Wyświetlenie wyników dla zbioru testowego
print("\nWyniki klasyfikacji:")
print(classification_report(true_labels, predictions))



Wyniki klasyfikacji:
              precision    recall  f1-score   support

           0       0.00      0.00      0.00        37
           1       0.58      0.82      0.68        74
           2       0.00      0.00      0.00         5
           3       0.29      0.42      0.34        12

    accuracy                           0.52       128
   macro avg       0.22      0.31      0.26       128
weighted avg       0.36      0.52      0.43       128



In [52]:

# Predykcje dla przykładowych zdań ze zbioru testowego
label_mapping = {0: 'negatywny', 1: 'neutralny', 2: 'pozytywny', 3: 'inne'}

def predict_sentence(sentence_words):
    model.eval()
    with torch.no_grad():
        inputs = tokenizer(
            sentence_words,
            is_split_into_words=True,
            return_tensors='pt',
            padding=True,
            truncation=True,
            max_length=MAX_LEN
        ).to(device)
        
        outputs = model(**inputs)
        predictions = torch.argmax(outputs.logits, dim=2)
        
        word_predictions = []
        word_ids = inputs.word_ids()
        
        current_word = None
        current_predictions = []
        
        for token_idx, word_idx in enumerate(word_ids):
            if word_idx is None:
                continue
            if word_idx != current_word:
                if current_word is not None:
                    # Wybierz najczęstszą predykcję dla słowa
                    word_predictions.append(max(set(current_predictions), key=current_predictions.count))
                current_word = word_idx
                current_predictions = []
            current_predictions.append(predictions[0][token_idx].item())
        
        # Dodaj ostatnie słowo
        if current_predictions:
            word_predictions.append(max(set(current_predictions), key=current_predictions.count))
            
        return word_predictions


In [53]:

print("\nPrzykładowe predykcje dla zdań ze zbioru testowego:")
for i in range(min(3, len(test_texts))):  # Pokazujemy pierwsze 3 zdania
    sentence = test_texts[i]
    predictions = predict_sentence(sentence)
    
    print(f"\nZdanie {i+1}:")
    for word, pred in zip(sentence, predictions):
        pred_label = label_mapping[pred]
        print(f"Słowo: {word:15} Predykcja: {pred_label}")



Przykładowe predykcje dla zdań ze zbioru testowego:

Zdanie 1:
Słowo: Jakość          Predykcja: neutralny
Słowo: i               Predykcja: inne
Słowo: praktyczność    Predykcja: neutralny
Słowo: wykonania       Predykcja: neutralny
Słowo: tego            Predykcja: neutralny
Słowo: trymera         Predykcja: neutralny
Słowo: pozostawia      Predykcja: neutralny
Słowo: naprawdę        Predykcja: neutralny
Słowo: wiele           Predykcja: neutralny
Słowo: do              Predykcja: inne
Słowo: życzenia        Predykcja: neutralny
Słowo: O               Predykcja: inne
Słowo: golarce         Predykcja: neutralny
Słowo: w               Predykcja: neutralny
Słowo: tym             Predykcja: neutralny
Słowo: zestawie        Predykcja: neutralny
Słowo: nie             Predykcja: neutralny
Słowo: warto           Predykcja: neutralny
Słowo: nawet           Predykcja: neutralny
Słowo: wspominać       Predykcja: neutralny
Słowo: Lepiej          Predykcja: neutralny
Słowo: od              Pred